<center><h1>Testing Notebook</h1></center>


## Loading Entities:

 - Power System
 - Ant Colony

In [ ]:
from acopoweropt import system, colony

PSystem = system.PowerSystem(name='s10')
Colony = colony.Colony(n_ants=5,phr_evp_rate=0.25)

## Testing Power System

In [ ]:
# Randomly select an operation
operation = PSystem.sample_operation()

# Solve system
solution = PSystem.solve(operation=operation)

## Testing Colony

In [9]:
# main.py
import pandas as pd
import random

from acopoweropt import colony, system

PSystem = system.PowerSystem(name='s15')
Colony = colony.PowerColony(n_ants=20,
                            pheromone_evp_rate={'worst': 0.4, 'mean': 0.25, 'best': 0.05},
                            power_system=PSystem)

n_iter = 150

In [10]:
for i in range(n_iter):
    # Getting last taken paths
    paths = Colony.paths.query("iteration == {i}".format(i=i))

    # Evaporate Pheromone
    Colony.evaporate_pheromone(paths=paths, power_system=PSystem)

    # Selecting Ants (80/20) to follow or not the pheromone paths
    taken_paths = []
    for ant in range(1, Colony.n_ants + 1):
        decision = random.random()
        if decision <= 0.8:
            # ANT Follows Path
            operative_zones = Colony.choose_path()
            existing_path = paths.query("path == '{}'".format(",".join([str(n) for n in operative_zones])))
            # Check if chosen path was already calculated:
            if existing_path.values.size != 0:
                solution = existing_path.reset_index().loc[0]
                solution['ant'] = ant
                solution['iteration'] = i+1
                solution = solution.to_frame().T.set_index('ant')
                taken_paths.append(solution)
            else:
                operation = PSystem.get_operation(operative_zones=operative_zones)
                solution = pd.DataFrame(
                    [colony.seek_food(ant=ant, iteration=i+1, operation=operation, power_system=PSystem)]
                ).set_index('ant')
                taken_paths.append(solution)

        else:
            # ANT Sets to new path
            operation = PSystem.sample_operation()
            solution = pd.DataFrame(
                [colony.seek_food(ant=ant, iteration=i+1, operation=operation, power_system=PSystem)]
            ).set_index('ant')
            taken_paths.append(solution)

    # Updating Paths
    new_paths = pd.concat(taken_paths).sort_index()
    Colony.paths = Colony.paths.append(new_paths)

    # Updating Pheromone
    Colony.update_pheromone(paths=new_paths)
    Colony.pheromone_history.update({i + 1: Colony.pheromone})
    print('Iter: {}'.format(i+1))
    print(Colony.pheromone)

Iter: 1
           1         2         3         4         5         6         7   \
opz                                                                         
1    0.830778  0.221607  0.830778  0.830778  0.129896  0.188642  0.830778   
2    0.000000  0.191738  0.000000  0.000000  0.268948  0.093806  0.000000   
3    0.000000  0.146968  0.000000  0.000000  0.063377  0.314323  0.000000   
4    0.000000  0.191660  0.000000  0.000000  0.279681  0.158755  0.000000   

           8         9         10        11        12        13        14  \
opz                                                                         
1    0.830778  0.830778  0.830778  0.830778  0.109026  0.830778  0.830778   
2    0.000000  0.000000  0.000000  0.000000  0.342155  0.000000  0.000000   
3    0.000000  0.000000  0.000000  0.000000  0.250142  0.000000  0.000000   
4    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

           15  
opz            
1    0.830778  
2    0.000000  
3 

In [14]:
Colony.paths

,iteration,path,status,distance,tau
ant,,,,,
1,0,"1,1,1,1,3,2,1,1,1,1,1,1,1,1,1",optimal,32828.775007,0.00003
2,0,"1,3,1,1,1,4,1,1,1,1,1,1,1,1,1",optimal,32658.635253,0.000031
3,0,"1,3,1,1,3,2,1,1,1,1,1,1,1,1,1",optimal,32599.023843,0.000031
4,0,"1,3,1,1,3,3,1,1,1,1,1,2,1,1,1",optimal,32553.401295,0.000031
5,0,"1,3,1,1,1,4,1,1,1,1,1,3,1,1,1",optimal,32618.615523,0.000031
...,...,...,...,...,...
16,150,"1,2,1,1,4,4,1,1,1,1,1,2,1,1,1",optimal,32572.569314,0.000031
17,150,"1,3,1,1,4,4,1,1,1,1,1,2,1,1,1",optimal,32560.688418,0.000031
18,150,"1,4,1,1,3,3,1,1,1,1,1,3,1,1,1",optimal,32549.844293,0.000031


In [ ]:
paths = Colony.paths[0].query("iteration == {i}".format(i=0))
solution = paths.query("path == '{}'".format('1,2,1,1,3,1,1,1,1,1,1,3,1,1,1')).reset_index().loc[0]
solution['ant'] = 1
solution['iteration'] = 1
solution = solution.to_frame().T

In [2]:
l = []
for i in Colony.paths:
    paths = Colony.paths[0]
    paths['iter'] = i
    l.append(paths)
    
df = pd.concat()

{0:                                               path   status      distance  \
 ant                                                                         
 1    [1, 3, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1]  optimal  32658.635253   
 2    [1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]  optimal  32812.154094   
 3    [1, 2, 1, 1, 3, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1]  optimal  32653.504720   
 4    [1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 2, 1, 1, 1]  optimal  32640.399291   
 5    [1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]  optimal  32767.297929   
 
           tau  
 ant            
 1    0.000031  
 2    0.000030  
 3    0.000031  
 4    0.000031  
 5    0.000031  }

In [4]:
optimal = [1,3,1,1,3,4,1,1,1,1,1,2,1,1,1]
test = [1,2,1,1,2,3,1,1,1,1,1,2,1,1,1]

operation = PSystem.get_operation(operative_zones=test)

solution = PSystem.solve(operation=operation)

pd.DataFrame([Colony.seek_food(ant=1, operation=operation, PowerSystem=PSystem)]).set_index('ant')

,path,status,distance,tau
ant,,,,
1,"1,2,1,1,2,3,1,1,1,1,1,2,1,1,1",optimal,32702.10893,0.000031
